In [31]:
# Installing dependencies
import numpy as np
import pandas as pd
from dLoader import DataLoader, BuySell, generate_df

In [32]:
def softmax(x):
    # Calculate softmax of x
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [33]:
class MACross:
    # Moving Average Fast Slow Crossover Strategy
    # Check Fast Moving Average against Slow Moving Average
    # Also checking Current Volume Average is above 
    # Volumne Moving Average
    def __init__(self, fast=3, slow=6):
        self.fast=3
        self.slow=6
    
    def check_ma_above(self, df):
        # Create Moving Averages for Price and Volume
        ndf = self.get_ma(df)
        # Check if price and volume is above Moving Averages
        if ndf['MA-Above'].iloc[-1] and ndf['MA-Vol-Above'].iloc[-1]:
            return True
        return False

    def get_ma(self, df):
        # MA Column Creations
        df['Fast'] = df['Close'].rolling(self.fast).mean()
        df['Slow'] = df['Low'].rolling(self.slow).mean()
        df['MA-Vol'] = df['Volume'].rolling(self.slow).mean()
        df['MA-Above'] = df['Fast'] > df['Slow']
        df['MA-Vol-Above'] = df['Volume'] > df['MA-Vol']
        return df


In [55]:
class GapTable:
    # Using data to create a Gap Percentage Table
    # Table contain Counts, Percentage, and Softmax
    def __init__(self, 
                 data, step=.01, a_col='Low', a_shift=None,
                 b_col='High', b_shift=None):
        self.update_prob(data=data, step=step,
                         a_col=a_col, a_shift=a_shift,
                         b_col=b_col, b_shift=b_shift)
    
    def topk(self, k=5, col='Softmax'):
        # Return top K largest Gap Ranges
        return self.prob.nlargest(k, columns=col)
    
    def update_prob(self, data=None, step=None, a_col=None, 
                    a_shift=None, b_col=None, b_shift=None):
        self.step = self.step if step is None else step
        self.data = self.data if data is None else data
        self.a = self.change(col=a_col, s=a_shift)
        self.b = self.change(col=b_col, s=b_shift)
        # Create Probability table
        self.get_pct()
        self.create_range()
        self.create_probability_df()
    
    def create_probability_df(self):
        # Create Probability Table base on Gap percentage range
        dic = {}
        for i, pair in enumerate(self.pairs):
            # Check if Gap percentage is within range
            isin = (self.hl >= pair[0]) & (self.hl < pair[1])
            # Assign the total counts into dictionary
            dic[i] = {'A': pair[0], 
                      'B': pair[1],
                      'Counts': len(self.hl[isin])}
        # Turn dictionary into DataFrame
        self.prob = pd.DataFrame.from_dict(dic, orient='index')
        # Calculate Percentage
        self.prob['PCT'] = self.prob['Counts'] / self.prob['Counts'].sum()
        # Calculate Softmax
        self.prob['Softmax'] = softmax(self.prob['PCT'])

    def create_range(self):
        # Getting Gap percentage range
        total = self.hl.max() - self.hl.min()
        drange = np.arange(.0, np.round(total, 2), self.step) + np.round(self.hl.min(), 2)
        self.pairs = np.stack([drange[:-1], drange[1:]]).T
    
    def change(self, col, data=None, s=None):
        # Change different between data columns
        data = self.data if data is None else data
        if s is not None:
            return data[col].shift(s)
        return data[col]

    def get_pct(self):
        # Get percentage different
        self.hl = self.a / self.b - 1

In [47]:
class LookBackTest:
    def __init__(self, fast=3, slow=6, capital=1000, max_share=100):
        # MA Calculation Class
        self.mc = MACross(fast, slow)
        self.lookback = slow + 1
        # Buy Sell
        self.bs = BuySell(capital, max_share)
    
    def recur_lookback(self, data, i=None):
        # Recursion for trading BackTest
        i = self.lookback if i is None else i
        if (i + 1) == len(data):
            self.update_trades(data, i, sell_all=True)
            # Showing Result
            self.bs.show_results()
            return
        # Update trade data
        self.update_trades(data, i)
        # Recursion
        self.recur_lookback(data, i+1)
    
    def update_trades(self, data, idx, sell_all=False):
        # Update record of trades
        date = data.index[idx]
        hPrice, lPrice, cPrice = data.loc[date, ['High', 'Low', 'Close']]
        # Logic for Buying and Selling
        above_ma = self.is_ma_above(data, date)
        if sell_all:
            if self.bs.is_holding:
                self.bs.sell(cPrice)
        else:
            # Buy when not holding
            if not self.bs.is_holding:
                if above_ma:
                    self.bs.buy(lPrice * .99)
            else:
                # Sell when holding
                if not above_ma:
                    self.bs.sell(hPrice * .99)
    
    def is_ma_above(self, data, date):
        # Check if the Price and Volume is above Moving Average
        lookback_data = self.lookback_data(data, date)
        return self.mc.check_ma_above(lookback_data)

    def lookback_data(self, df, current_date):
        # Getting Lookback data base on current date
        lookback_range = pd.date_range(end=current_date, 
                                       periods=self.lookback*2, 
                                       freq='B')
        return df.loc[df.index.isin(lookback_range[: -1])].copy()

In [25]:
# Load data
stock_data = DataLoader('AAPL')

In [26]:
# Get a period of data for testing
data = stock_data.get_data('2018-01-01', '2019-12-31')
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,42.540001,43.075001,42.314999,43.064999,41.188156,102223600
2018-01-03,43.132500,43.637501,42.990002,43.057499,41.180992,118071600
2018-01-04,43.134998,43.367500,43.020000,43.257500,41.372272,89738400
2018-01-05,43.360001,43.842499,43.262501,43.750000,41.843311,94640000
2018-01-08,43.587502,43.902500,43.482498,43.587502,41.687893,82271200
...,...,...,...,...,...,...
2019-12-24,71.172501,71.222504,70.730003,71.067497,70.027313,48478800
2019-12-26,71.205002,72.495003,71.175003,72.477501,71.416672,93121200
2019-12-27,72.779999,73.492500,72.029999,72.449997,71.389572,146266000


In [61]:
# Gap probability Table with Past Data
pdata = stock_data.get_data('2017-01-01', '2017-12-31')
lowHighGap = GapTable(pdata, step=.01, a_col='Low', b_col='High', b_shift=3)
highLowGap = GapTable(pdata, step=.01, a_col='High', b_col='Low', b_shift=3)

In [62]:
highLowGap.prob

,A,B,Counts,PCT,Softmax
0,-0.05,-0.04,1,0.004049,0.066490
1,-0.04,-0.03,1,0.004049,0.066490
2,-0.03,-0.02,1,0.004049,0.066490
3,-0.02,-0.01,10,0.040486,0.068957
4,-0.01,0.00,19,0.076923,0.071516
5,0.00,0.01,42,0.170040,0.078495
6,0.01,0.02,76,0.307692,0.090079
7,0.02,0.03,51,0.206478,0.081408
8,0.03,0.04,25,0.101215,0.073275
9,0.04,0.05,8,0.032389,0.068401


In [64]:
lowHighGap.prob

,A,B,Counts,PCT,Softmax
0,-0.09,-0.08,1,0.004082,0.066528
1,-0.08,-0.07,1,0.004082,0.066528
2,-0.07,-0.06,2,0.008163,0.066800
3,-0.06,-0.05,2,0.008163,0.066800
4,-0.05,-0.04,6,0.024490,0.067899
5,-0.04,-0.03,20,0.081633,0.071892
6,-0.03,-0.02,22,0.089796,0.072482
7,-0.02,-0.01,47,0.191837,0.080268
8,-0.01,0.00,65,0.265306,0.086388
9,0.00,0.01,51,0.208163,0.081589


In [131]:
lb_test = LookBackTest()

In [132]:
lb_test.recur_lookback(data)

Average gain of $1.19 per share after 82.0 trades
and have average gain percentage of 2.42%
Has a 91.46% of good trades
Test Ending Capital: $6800.49 base on original capital of $1000.00
With 580.05% Capital Gain
